In [1]:
import cvxpy as cp
import numpy as np

In [2]:
t = 5

In [3]:
rng = np.random.randint(10, 100, size=t)

In [4]:
very_big_number = np.sum(rng)

In [5]:
k = cp.Parameter(name='k', nonneg=True)
h = cp.Parameter(name='h', nonneg=True)
c = cp.Parameter(name='c', nonneg=True)
Dt = cp.Parameter((t,), name='Dt', nonneg=True)
xt = cp.Variable((t,), nonneg=True)
It = cp.Variable((t+1,))
Qt = cp.Variable((t,))
ut = cp.Parameter((t,), name='ut', nonneg=True)
vt = cp.Parameter((t,), name='vt',  nonneg=True)
wt = cp.Parameter((t+1,), name='wt', nonneg=True)
zt = cp.Parameter((t,), name='zt', nonneg=True)
lambda1 = cp.Parameter((t,), name='l1')
lambda2 = cp.Parameter(name='l2')
lambda3 = cp.Parameter(name='l3')
lambda4 = cp.Parameter((t,), name='l4')
lambda5 = cp.Parameter((t,), name='l5')
lambda6 = cp.Parameter((t+1,), name='l6')
lambda7 = cp.Parameter((t,), name='l6')
mu = cp.Parameter(name='mu', nonneg=True)

In [6]:
augmented_lagrangian = cp.sum(k * xt + h * It[1:] + c * Qt) \
    + cp.sum(cp.multiply(lambda1, It[1:] - It[:-1] - Qt + Dt)) + lambda2 * It[0] + lambda3 * It[-1]\
    + cp.sum(
        cp.multiply(lambda4, xt - ut) + cp.multiply(lambda5, Qt - vt)
        + cp.multiply(lambda7, very_big_number * ut - Qt - zt)
    ) + cp.sum(cp.multiply(lambda6, It - wt)) + 1 / (2*mu) * (cp.sum_squares(It[1:] - It[:-1] - Qt + Dt) + It[0] ** 2 + It[-1] ** 2 
                      + cp.sum_squares(xt - ut) + cp.sum_squares(Qt - vt) + cp.sum_squares(It - wt) + cp.sum_squares(very_big_number * ut - Qt - zt)
                    )

In [7]:
Dt.value = rng
k.value = 100000
h.value = 1
c.value = 1
xt.value = np.ones(t) * 0.5
It.value = np.zeros(t+1)
Qt.value = np.zeros(t)
ut.value = np.ones(t)
vt.value = np.zeros(t)
wt.value = np.zeros(t+1)
zt.value = np.zeros(t)
lambda1.value = np.zeros(t)
lambda2.value = 0
lambda3.value = 0
lambda4.value = np.zeros(t)
lambda5.value = np.zeros(t)
lambda6.value = np.zeros(t+1)
lambda7.value = np.zeros(t)
mu.value = 1000

In [8]:
subprob_main_var = cp.Problem(cp.Minimize(augmented_lagrangian), constraints=[xt <= 1])

In [9]:
subprob_main_var.is_dcp(dpp=True)

False

In [10]:
n_iter = 200
gamma = 0.9
for i in range(n_iter):
    subprob_main_var.solve(warm_start=True)
    # update uz
    prox_ueq0 = np.where((lambda7 * mu - Qt).value >= 0, 
                         1 / (2 * mu.value) * ((lambda7.value * mu.value)**2 + xt.value**2) 
                         - lambda7.value * (lambda7.value * mu.value - Qt.value),
                         1 / (2*mu.value) * (Qt.value ** 2 + xt.value ** 2))
    prox_ueq1 = np.where((lambda7 * mu - Qt + very_big_number).value >= 0, 
                         1 / (2 * mu.value) * ((lambda7.value * mu.value)**2 + (xt.value - 1)**2) 
                         - lambda4.value - lambda7.value * (lambda7.value * mu.value - Qt.value + very_big_number),
                         1 / (2*mu.value) * ((Qt.value - very_big_number) ** 2 + (xt.value - 1) ** 2) - lambda4.value)
    ut.value = np.where(prox_ueq0 < prox_ueq1, 0, 1)
    zt.value = np.where(prox_ueq0 < prox_ueq1, np.where((lambda7 * mu - Qt).value >= 0, (lambda7 * mu - Qt).value, 0), 
                        np.where((lambda7 * mu - Qt + very_big_number).value >= 0, (lambda7 * mu - Qt + very_big_number).value, 0))
    # update vw
    vt.value = np.where((Qt.value + mu.value * lambda5.value) > 0, Qt.value + mu.value * lambda5.value, 0)
    wt.value = np.where((It.value + mu.value * lambda6.value) > 0, It.value + mu.value * lambda6.value, 0)
    # update lambda
    lambda1.value = lambda1.value + (It[1:] - It[:-1] - Qt + Dt).value / mu.value
    lambda2.value = lambda2.value + It[0].value / mu.value
    lambda3.value = lambda3.value + It[-1].value / mu.value
    lambda4.value = lambda4.value + (xt - ut).value / mu.value
    lambda5.value = lambda5.value + (Qt - vt).value / mu.value
    lambda6.value = lambda6.value + (It - wt).value / mu.value
    lambda7.value = lambda7.value + (very_big_number * ut - Qt - zt).value / mu.value
    # update mu
    mu.value *= gamma

c:\Users\Tanaphoom\AppData\Local\Programs\Python\Python311\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:213: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)


In [11]:
xt.value

array([1., 1., 1., 0., 1.])

In [12]:
Dt.value

array([86, 67, 52, 18, 68])

In [13]:
Qt.value

array([ 8.59999999e+01,  6.70000003e+01,  6.99999998e+01, -2.22507248e-08,
        6.80000001e+01])

In [14]:
It.value

array([ 3.27840431e-08,  1.07869483e-08,  3.56529902e-07,  1.80000001e+01,
       -4.80018798e-10,  2.24907343e-08])

In [15]:
ut.value

array([1, 1, 1, 0, 1])

In [16]:
vt.value

array([85.99999991, 67.00000027, 69.99999975,  0.        , 68.00000007])

In [17]:
wt.value

array([2.05304420e-08, 0.00000000e+00, 3.56529902e-07, 1.80000001e+01,
       0.00000000e+00, 0.00000000e+00])

In [18]:
zt.value

array([205.00000009, 223.99999973, 221.00000025,   0.        ,
       222.99999993])

In [19]:
subprob_main_var.value

400309.0000004702